In [1]:
import pandas as pd

In [2]:
df = pd.read_pickle('../data/data_w_descs.pkl')
df.head(2)

,CASE_ENQUIRY_ID,OPEN_DT,TARGET_DT,CLOSED_DT,OnTime_Status,CASE_STATUS,CLOSURE_REASON,CASE_TITLE,SUBJECT,REASON,...,Property_Type,Property_ID,LATITUDE,LONGITUDE,Source,Geocoded_Location,case_enquiry_id,description,specific_location,title
0,101000958209,11/01/2013 09:27:19 AM,11/15/2013 09:27:19 AM,11/27/2013 10:15:45 AM,OVERDUE,Closed,Case Closed Case Resolved,Sign Repair,Transportation - Traffic Division,Signs & Signals,...,Intersection,4053.0,42.3537,-71.0580,Constituent Call,"(42.3537, -71.058)",1.010010e+11,Public safety sign: No | Type of sign: no stop...,None,Damaged Sign at Intersection of Devonshire St ...
1,101001243239,01/06/2015 03:26:49 PM,01/21/2015 03:26:00 PM,01/12/2015 10:49:02 PM,ONTIME,Closed,Case Closed Case Resolved replaced LED. jouber...,outage///printed,Public Works Department,Street Lights,...,Address,63245.0,42.3594,-71.0587,Employee Generated,"(0, 0)",1.010012e+11,Electric or gas light: Electric,None,"Street Lights at 0 Geneva Ave, Dorchester"


In [3]:
df.shape

(905650, 37)

## Adding census tract to df

In [4]:
import shapegeocode

In [5]:
def get_tract_block_group(lat, long_, shp_obj):
    d = gc.geocode(lat, long_)
    return d['TRACTCE'] + d['BLKGRPCE']

In [6]:
gc = shapegeocode.geocoder('../data/shape-files/cb_2015_25_bg_500k.shp')

In [7]:
df.shape

(905650, 37)

In [8]:
df['tract_and_block_group'] = df[['LATITUDE', 'LONGITUDE']].apply(lambda row: get_tract_block_group(row['LATITUDE'], row['LONGITUDE'], gc), axis=1)

KeyboardInterrupt: 

In [ ]:
df.head(1).T

In [ ]:
df.shape

In [ ]:
df.tract_and_block_group.isnull().sum()

## Now adding census data to df

In [ ]:
df_census = pd.read_pickle('../data/census_data_aggregated.pkl')
df_census.head(1).T

In [ ]:
df_final = df.merge(df_census, on='tract_and_block_group', how='left')
df_final.head(1).T

In [ ]:
df.shape, df_census.shape, df_final.shape

In [ ]:
from datetime import datetime
to_datetime = lambda d: datetime.strptime(d, '%m/%d/%Y %H:%M:%S %p') if type(d) != float else pd.np.NaN

In [ ]:
df['OPEN_DT'] = df['OPEN_DT'].apply(lambda datetime_text: to_datetime(datetime_text))
df['CLOSED_DT'] = df['CLOSED_DT'].apply(lambda datetime_text: to_datetime(datetime_text))
df['TARGET_DT'] = df['TARGET_DT'].apply(lambda datetime_text: to_datetime(datetime_text))

In [ ]:
df_final.to_hdf('../data/data_w_descs_and_census.h5', key='data_w_descs_and_census', format='fixed')

## Let's look at the rows where there was no census tract + block in Suffolk county that matched that of the latlong